<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/YOLOv3_Dec_seg_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make

In [35]:
!apt-get install libopencv-dev  # Instala dependências do OpenCV
!make clean && make  # Recompile após instalação

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/represen

In [ ]:
!sudo apt-get install tree

In [34]:
!mkdir -p /content/backup

In [ ]:
!mkdir -p /content/coco/images
!mkdir -p /content/coco/annotations

In [5]:
# Baixar val2017 (1GB)
!wget -c http://images.cocodataset.org/zips/val2017.zip -P /content/coco/images

# Baixar test2017
!wget -c http://images.cocodataset.org/zips/test2017.zip -P /content/coco/images

--2025-02-15 21:52:57--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.209.81, 3.5.22.67, 16.182.73.233, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.209.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘/content/coco/images/val2017.zip’

val2017.zip         100%[===================>] 777.80M  19.1MB/s    in 43s     

2025-02-15 21:53:41 (17.9 MB/s) - ‘/content/coco/images/val2017.zip’ saved [815585330/815585330]

--2025-02-15 21:53:41--  http://images.cocodataset.org/zips/test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.195.89, 3.5.28.57, 3.5.29.71, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.195.89|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6646970404 (6.2G) [application/zip]
Saving to: ‘/content/coco/images/test2017.zip’

test

In [6]:
# Criar diretório para anotações
!mkdir -p /content/coco/annotations

# Baixar anotações (train + val)
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/coco/annotations

--2025-02-15 22:00:24--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.217.139, 52.216.40.57, 52.216.250.196, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.217.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/content/coco/annotations/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  20.5MB/s    in 13s     

2025-02-15 22:00:38 (18.2 MB/s) - ‘/content/coco/annotations/annotations_trainval2017.zip’ saved [252907541/252907541]



In [ ]:
!unzip /content/coco/images/val2017.zip -d /content/coco/images
!rm /content/coco/images/val2017.zip

!unzip /content/coco/images/test2017.zip -d /content/coco/images
!rm /content/coco/images/test2017.zip

In [8]:
# Baixar imagens de treino (18GB)
!wget -c http://images.cocodataset.org/zips/train2017.zip -P /content/coco/images

--2025-02-15 22:03:54--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.200.201, 52.217.203.89, 52.216.206.251, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.200.201|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘/content/coco/images/train2017.zip’

train2017.zip       100%[===================>]  18.01G  20.0MB/s    in 16m 35s 

2025-02-15 22:20:29 (18.5 MB/s) - ‘/content/coco/images/train2017.zip’ saved [19336861798/19336861798]



In [ ]:
# Descompactar imagens
!unzip /content/coco/images/train2017.zip -d /content/coco/images
!rm /content/coco/images/train2017.zip

# Descompactar anotações
!unzip /content/coco/annotations/annotations_trainval2017.zip -d /content/coco/annotations
!rm /content/coco/annotations/annotations_trainval2017.zip

In [12]:
# Mova todos os JSONs para /content/coco/annotations/
!mv /content/coco/annotations/annotations/*.json /content/coco/annotations/
!rm -rf /content/coco/annotations/annotations  # Delete a subpasta vazia

In [13]:
!ls /content/coco/annotations/

captions_train2017.json  instances_train2017.json  person_keypoints_train2017.json
captions_val2017.json	 instances_val2017.json    person_keypoints_val2017.json


In [14]:
!tree -d /content/coco/

/content/coco/
├── annotations
└── images
    ├── test2017
    ├── train2017
    └── val2017

5 directories


In [17]:
!tree -d /content/coco/images/

/content/coco/images/
├── test2017
├── train2017
└── val2017

3 directories


In [15]:
import os
import shutil
from pycocotools.coco import COCO

def process_coco_split(splits: list, categories: list, max_samples_per_split: int = None):
    """
    Processa splits do COCO (train, val, test), filtrando por categorias e convertendo para YOLO.
    """
    # Criar diretório principal
    os.makedirs('/content/coco/filtered/', exist_ok=True)

    # Criar arquivo de classes
    with open('/content/coco/filtered/obj.names', 'w') as f:
        for cat in categories:
            f.write(cat + '\n')

    for split in splits:
        print(f"\nProcessando split: {split}")

        # Caminhos das imagens
        img_source_dir = f'/content/coco/images/{split}2017'
        img_dest_dir = f'/content/coco/filtered/{split}/images'
        os.makedirs(img_dest_dir, exist_ok=True)

        # Processar apenas train/val (test não tem anotações)
        if split != 'test':
            # Carregar anotações
            coco = COCO(f'/content/coco/annotations/instances_{split}2017.json')
            cat_ids = coco.getCatIds(catNms=categories)
            img_ids = coco.getImgIds(catIds=cat_ids)

            # Limitar amostras se especificado
            if max_samples_per_split:
                img_ids = img_ids[:max_samples_per_split]

            images = coco.loadImgs(img_ids)

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

            # Criar diretório de labels
            ann_dest_dir = f'/content/coco/filtered/{split}/labels'
            os.makedirs(ann_dest_dir, exist_ok=True)

            # Converter anotações para YOLO
            for img in images:
                ann_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids)
                annotations = coco.loadAnns(ann_ids)

                label_path = os.path.join(ann_dest_dir, img['file_name'].replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        x, y, w, h = ann['bbox']
                        img_w, img_h = img['width'], img['height']

                        cx = (x + w/2) / img_w
                        cy = (y + h/2) / img_h
                        nw = w / img_w
                        nh = h / img_h

                        class_id = categories.index(coco.loadCats(ann['category_id'])[0]['name'])
                        f.write(f"{class_id} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

        # Processar test (sem anotações)
        else:
            # Listar todas as imagens do diretório test2017
            images = [{'file_name': f} for f in os.listdir(img_source_dir)]
            if max_samples_per_split:
                images = images[:max_samples_per_split]

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

# Uso
process_coco_split(
    splits=['train', 'val', 'test'],
    categories=['boat', 'person'],
    max_samples_per_split=100
)


Processando split: train
loading annotations into memory...
Done (t=20.65s)
creating index...
index created!

Processando split: val
loading annotations into memory...
Done (t=0.66s)
creating index...
index created!

Processando split: test


In [16]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2025-02-15 22:26:42--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  18.9MB/s    in 14s     

2025-02-15 22:26:56 (17.2 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [33]:
!which yolov3.weights

In [24]:
!cat /content/coco/filtered/obj.names

boat
person


In [26]:
!echo -e "classes = 2\ntrain = /content/coco/filtered/train.txt\nvalid = /content/coco/filtered/val.txt\nnames = /content/coco/filtered/obj.names\nbackup = /content/backup" > /content/coco/filtered/obj.data

In [27]:
!cat /content/coco/filtered/obj.data

classes = 2
train = /content/coco/filtered/train.txt
valid = /content/coco/filtered/val.txt
names = /content/coco/filtered/obj.names
backup = /content/backup


In [28]:
import os

# Para train.txt
with open('/content/coco/filtered/train.txt', 'w') as f:
    train_img_dir = '/content/coco/filtered/train/images'
    for img_name in os.listdir(train_img_dir):
        f.write(f'{train_img_dir}/{img_name}\n')

# Para val.txt
with open('/content/coco/filtered/val.txt', 'w') as f:
    val_img_dir = '/content/coco/filtered/val/images'
    for img_name in os.listdir(val_img_dir):
        f.write(f'{val_img_dir}/{img_name}\n')

In [29]:
# Verifique se existem e contêm caminhos das imagens
!ls /content/coco/filtered/train.txt /content/coco/filtered/val.txt

# Exemplo do conteúdo de train.txt (primeiras 2 linhas)
!head -n 2 /content/coco/filtered/train.txt

/content/coco/filtered/train.txt  /content/coco/filtered/val.txt
/content/coco/filtered/train/images/000000032818.jpg
/content/coco/filtered/train/images/000000549127.jpg


In [30]:
!tree /content/coco/filtered -L 2

/content/coco/filtered
├── obj.data
├── obj.names
├── test
│   └── images
├── train
│   ├── images
│   └── labels
├── train.txt
├── val
│   ├── images
│   └── labels
└── val.txt

8 directories, 4 files


In [22]:
# Cria cópia do cfg original
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

# Aplica modificações
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_custom.cfg
!sed -i 's/classes=80/classes=2/' cfg/yolov3_custom.cfg
!sed -i 's/filters=255/filters=21/' cfg/yolov3_custom.cfg

In [36]:
!./darknet detector train /content/coco/filtered/obj.data cfg/yolov3_custom.cfg yolov3.weights -dont_show -map

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
 Prepare additional network for mAP calculation...
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x

In [37]:
!tree /content/coco/filtered/train -L 2

/content/coco/filtered/train
├── images
│   ├── 000000016773.jpg
│   ├── 000000024609.jpg
│   ├── 000000032818.jpg
│   ├── 000000032965.jpg
│   ├── 000000041199.jpg
│   ├── 000000041247.jpg
│   ├── 000000049165.jpg
│   ├── 000000049175.jpg
│   ├── 000000049402.jpg
│   ├── 000000065755.jpg
│   ├── 000000065894.jpg
│   ├── 000000073753.jpg
│   ├── 000000073826.jpg
│   ├── 000000073898.jpg
│   ├── 000000073976.jpg
│   ├── 000000090359.jpg
│   ├── 000000090476.jpg
│   ├── 000000098345.jpg
│   ├── 000000098601.jpg
│   ├── 000000098636.jpg
│   ├── 000000106755.jpg
│   ├── 000000106865.jpg
│   ├── 000000114732.jpg
│   ├── 000000114879.jpg
│   ├── 000000139440.jpg
│   ├── 000000155763.jpg
│   ├── 000000155885.jpg
│   ├── 000000163929.jpg
│   ├── 000000172196.jpg
│   ├── 000000172271.jpg
│   ├── 000000172369.jpg
│   ├── 000000180354.jpg
│   ├── 000000196672.jpg
│   ├── 000000204805.jpg
│   ├── 000000205035.jpg
│   ├── 000000213357.jpg
│   ├── 000000213381.jpg
│   ├── 000000229522.jpg
│   ├── 00